<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/signature_verification_densenet_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Keras libraries and packages
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import NASNetMobile
from keras.applications.densenet import DenseNet201
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 224, 224



In [37]:
#Load the pretrained Network
densenet_model=DenseNet201(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height,img_width,3), pooling=None, classes=1000)
print("pretrained Network is loaded")



74842112/74836368 [==============================] - 7s 0us/step
pretrained Network is loaded


In [38]:
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [39]:
# Freeze the layers
for layer in densenet_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")
    

Pretrained layers are freezed


In [40]:
model = Sequential()
model.add(densenet_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(1, activation='sigmoid', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [41]:
train_data_dir ='/content/drive/My Drive/Dataset1/training_set'
val_data_dir ='/content/drive/My Drive/Dataset1/testing_set'
nb_epochs = 35
file_path='/content/drive/My Drive/Colab Notebooks/Signature Verification Densenet Model/densenet_model_weights.hdf5'
print("Input parameters are assigned")

Input parameters are assigned


In [42]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
# Imaga data generation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False,class_mode='binary')


Found 2112 images belonging to 2 classes.
Found 544 images belonging to 2 classes.


In [45]:

callbacks = [ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, nb_epoch=nb_epochs, validation_data=validation_generator)

print('Training Completed!')



Epoch 1/35
66/66 [==============================] - 80s 1s/step - loss: 3.7249 - accuracy: 0.5365 - val_loss: 5.5591 - val_accuracy: 0.4926
Epoch 2/35
66/66 [==============================] - 52s 783ms/step - loss: 1.4772 - accuracy: 0.5431 - val_loss: 1.1526 - val_accuracy: 0.5074
Epoch 3/35
66/66 [==============================] - 54s 812ms/step - loss: 0.9802 - accuracy: 0.6780 - val_loss: 0.1768 - val_accuracy: 0.5092
Epoch 4/35
66/66 [==============================] - 54s 813ms/step - loss: 1.4381 - accuracy: 0.6037 - val_loss: 1.3108 - val_accuracy: 0.4926
Epoch 5/35
66/66 [==============================] - 54s 811ms/step - loss: 0.9377 - accuracy: 0.6894 - val_loss: 0.8433 - val_accuracy: 0.6066
Epoch 6/35
66/66 [==============================] - 53s 797ms/step - loss: 1.2240 - accuracy: 0.6402 - val_loss: 2.9026 - val_accuracy: 0.4926
Epoch 7/35
66/66 [==============================] - 53s 810ms/step - loss: 0.7486 - accuracy: 0.7150 - val_loss: 0.8135 - val_accuracy: 0.5790
Ep

In [46]:
 # save model and architecture to single file
model.save('/content/drive/My Drive/Colab Notebooks/Signature Verification Densenet Model/densenet_model.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 7, 7, 1920)        18321984  
_________________________________________________________________
flatten_2 (Flatten)          (None, 94080)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12042368  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 30,372,673
Trainable params: 12,050,689
Non-trainable params: 18,321,984
_________________________________________________________________
Saved model to disk


In [47]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Colab Notebooks/Signature Verification Densenet Model/densenet_model.h5')
print("Model is Loaded")

Model is Loaded


In [49]:
# Intermediate Model construction
from keras.models import load_model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Intermediate model is created")


Intermediate model is created


In [50]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [51]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Colab Notebooks/Signature Verification Densenet Model/densenet_intermediate_model.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201_input (InputLaye (None, 224, 224, 3)       0         
_________________________________________________________________
densenet201 (Model)          (None, 7, 7, 1920)        18321984  
_________________________________________________________________
flatten_2 (Flatten)          (None, 94080)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12042368  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 30,372,608
Trainable params: 12,050,624
Non-trainable params: 18,321,984
_________________________________________________________________
Saved Intermediate model to disk


In [52]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Colab Notebooks/Signature Verification Densenet Model/densenet_intermediate_model.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [53]:
# Training Label feature identification(y_train)
 import numpy as np
 batch_size=32
 sample_count=2112
 features = np.zeros(shape=(2112, 64))  # Must be equal to the output of the convolutional base
 labels = np.zeros(shape=(2112))
 i = 0
 for inputs_batch, labels_batch in train_generator:
   features_batch = model.predict(inputs_batch)
   features[i * batch_size: (i + 1) * batch_size] = features_batch
   labels[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels.shape)

(2112,)


In [54]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train.shape)
print(labels_train)#identification of training Labels
features_train=features
print(features_train.shape)
print(features_train)

(2112, 64)
(2112, 1)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]
(2112, 64)
[[ 0.          4.45862675  0.         ...  0.          0.66358376
   0.        ]
 [ 0.          4.20041275  0.         ...  0.          0.25132447
   0.        ]
 [ 0.          4.43251467  0.         ...  0.          0.37281185
   0.        ]
 ...
 [ 0.         10.47244835  0.         ...  0.          4.63346052
   0.        ]
 [ 0.          4.82408237  0.         ...  0.          0.
   0.        ]
 [ 0.          5.99214649  0.         ...  0.          0.96784818
   0.        ]]


In [55]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train.shape)
print(labels_train)

(2112, 64)
(2112, 1)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [56]:
# identification of test labels
 import numpy as np
 batch_size=32
 sample_count=544
 features_test = np.zeros(shape=(544, 64))  # Must be equal to the output of the convolutional base
 labels_test = np.zeros(shape=(544))
 i = 0
 for inputs_batch, labels_batch in validation_generator:
   features_batch = model.predict(inputs_batch)
   features_test[i * batch_size: (i + 1) * batch_size] = features_batch
   labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels_test.shape)
 print(features_test.shape)
 print(labels_test)

(544,)
(544, 64)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [57]:
# #identification of testing Labels
print(features_test.shape)
print(features_test)
labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)

(544, 64)
[[ 0.          9.67008495  0.         ...  0.          1.78248954
   0.        ]
 [ 0.         13.65451622  0.         ...  0.          4.2400918
   0.        ]
 [ 0.          8.75526714  0.         ...  0.          3.01596475
   0.        ]
 ...
 [ 0.          8.75973415  0.         ...  0.          2.26981091
   0.        ]
 [ 0.          5.31555843  0.         ...  0.          0.50501955
   0.        ]
 [ 0.         10.54585743  0.         ...  0.          2.94071794
   0.        ]]
(544, 1)


In [58]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train,labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[154 114]
 [116 160]]
0.5772058823529411


In [59]:

# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))


[[187  81]
 [148 128]]
0.5790441176470589


In [60]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))



[[186  82]
 [182  94]]
0.5147058823529411


In [61]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[152 116]
 [149 127]]
0.5128676470588235


In [62]:

# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))


[[226  42]
 [184  92]]
0.5845588235294118


In [63]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[163 105]
 [156 120]]
0.5202205882352942


In [64]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[194  74]
 [158 118]]
0.5735294117647058


In [65]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Colab Notebooks/Signature Verification Densenet Model/densenet_model.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
17/17 [==============================] - 18s 1s/step
test loss, test acc: [3.0524814128875732, 0.5238970518112183]
